## Fluxo do notebook:
- Análise exploratória dos dados
    - Compreenção das variáveis (Tipos)
    - Análise descritiva
    - Detecção de outliers (histograma, boxplot)
    - Padrões e Relações (relação entre variáveis)
    - Avaliar a presença de dados faltantes (imputação, remoção, etc)
    - Verificação de suposições

##### Neg.

Após fazer uma limpeza e seleção das variáveis que poderão ser utilizadas no decorrer do projeto, iremos compreender melhor como é a sua distribuição e relação entre variáveis. Temos aqui dois dataframes, um relacionado a vendas de produtos e outro relacionado aos clientes em si e suas características.

##### Objetivo

Compreender os dados para a realização dos próximos processamentos, sendo eles: RFM, Sazonalidade e probabilidade de Churn. Será realizado também a Feature Engineer para melhorar os dados para o modelo. A estruturação dos testes para os processamentos citados será realizado em outro notebook com todo o processo explicado.

Abaixo temos as importações das bibliotecas que serão usadas, tanto para a análise, quanto para a utilização da env.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
dados_vendas_limpo = os.getenv("ENDERECO_DADOS_LIMPOS_VENDAS")
dados_clientes_limpo = os.getenv("ENDERECO_DADOS_LIMPOS_CLIENTES")
df_vendas_limpo = pd.read_csv(dados_vendas_limpo)
df_clientes_limpo = pd.read_csv(dados_clientes_limpo)

DataFrame:
- CustomerID: Identificador único do cliente.
- Churn: Indica se um cliente cancelou o serviço (1) ou se está ativo (0).
- Tenure: Duranção da permanência do cliente com a empresa. Em meses.
- PreferredLoginDevice: Dispositivo preferido do cliente para acesso à conta (exemplo, smartphone, desktop, etc.)
- PreferredPaymentMode: Modo de pagamento preferido pelo cliente (cartão de crédito, débito, etc.)
- Gender: Gênero do cliente (masculino, feminino ou outro).
- 

### Análise dos dados

- Análise univariada dos dados
- Análise multivariada dos dados

In [4]:
numericos = df_vendas_limpo.select_dtypes('number').columns.tolist()
categoricos = df_vendas_limpo.select_dtypes('object').columns.tolist()

print(f"Há {len(numericos)} dados numéricos, sendo eles:")
print(numericos)
print(f"\nHá {len(categoricos)} dados categóricos, sendo eles:")
print(categoricos)

Há 4 dados numéricos, sendo eles:
['InvoiceNo', 'Quantity', 'UnitPrice', 'CustomerID']

Há 4 dados categóricos, sendo eles:
['StockCode', 'Description', 'InvoiceDate', 'Country']


In [5]:
numericos2 = df_clientes_limpo.select_dtypes('number').columns.tolist()
categoricos2 = df_clientes_limpo.select_dtypes('object').columns.tolist()

print(f"Há {len(numericos2)} dados numéricos, sendo eles:")
print(numericos2)
print(f"\nHá {len(categoricos2)} dados categóricos, sendo eles:")
print(categoricos2)

Há 7 dados numéricos, sendo eles:
['CustomerID', 'Churn', 'Tenure', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'Complain', 'CouponUsed']

Há 5 dados categóricos, sendo eles:
['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'MaritalStatus', 'NomeCustomer']


In [6]:
for elemento in categoricos2:
  print(elemento)
  print('-'*40)
  print(f'Há {df_clientes_limpo[elemento].nunique()} valores únicos, sendo eles:')
  print(f'{df_clientes_limpo[elemento].value_counts(normalize=True)}\n')

PreferredLoginDevice
----------------------------------------
Há 4 valores únicos, sendo eles:
PreferredLoginDevice
Mobile Phone    0.483532
Computer        0.290256
Phone           0.216148
0               0.010064
Name: proportion, dtype: float64

PreferredPaymentMode
----------------------------------------
Há 7 valores únicos, sendo eles:
PreferredPaymentMode
Debit Card          0.407539
Credit Card         0.265529
E wallet            0.109531
UPI                 0.074206
COD                 0.065434
CC                  0.050735
Cash on Delivery    0.027027
Name: proportion, dtype: float64

Gender
----------------------------------------
Há 2 valores únicos, sendo eles:
Gender
Male      0.607502
Female    0.392498
Name: proportion, dtype: float64

MaritalStatus
----------------------------------------
Há 3 valores únicos, sendo eles:
MaritalStatus
Married     0.532446
Single      0.319468
Divorced    0.148087
Name: proportion, dtype: float64

NomeCustomer
-------------------------

### **Estatística descritiva dos dados numéricos**

A primeira é referente a tabela de vendas e a segunda é sobre a tabela de clientes.

In [7]:
df_vendas_limpo.describe().T.drop(labels=["CustomerID", "InvoiceNo"])

,count,mean,std,min,25%,50%,75%,max
Quantity,397884.0,12.988238,179.331775,1.000,2.00,6.00,12.00,80995.00
UnitPrice,397884.0,3.116488,22.097877,0.001,1.25,1.95,3.75,8142.75


**Quantidade Vendida:**
- A quantidade média de produtos vendidos por transação é de aproximadamente 13 unidades.
- A quantidade vendida varia significativamente, com um desvio padrão de 179 unidades, indicando que há transações com volumes muito grandes.
- A mediana é de 6 unidades, sugerindo que a maioria das transações envolve pequenas quantidades. O valor máximo é extremamente alto (80995 unidades), o que pode indicar grandes pedidos ou vendas em massa.
- 25% das transações envolvem até 2 unidades, enquanto 75% envolvem até 12 unidades.

**Preço Unitário:**
- O preço médio por unidade é de aproximadamente R$ 3.12.
- O desvio padrão é alto (R$ 22.10), indicando uma grande variação nos preços dos produtos.
- A mediana é de R$ 1.95, sugerindo que a maioria dos produtos tem preços baixos. O preço máximo é extremamente alto (R$ 8142.75), o que pode indicar produtos de luxo ou itens com preços especiais.
- 25% dos produtos são vendidos por até R$ 1.25, enquanto 75% são vendidos por até R$ 3.75.

In [8]:
df_clientes_limpo.describe().T.drop(labels=["CustomerID"])

,count,mean,std,min,25%,50%,75%,max
Churn,4372.0,0.168115,0.374011,0.0,0.0,0.0,0.0,1.0
Tenure,4166.0,-169.971675,1331.164566,-10000.0,2.0,9.0,16.0,61.0
NumberOfDeviceRegistered,4218.0,3.687055,1.027591,1.0,3.0,4.0,4.0,6.0
SatisfactionScore,4372.0,3.092406,1.390285,1.0,2.0,3.0,4.0,5.0
Complain,4372.0,0.285224,0.451573,0.0,0.0,0.0,1.0,1.0
CouponUsed,4173.0,1.773784,1.891626,0.0,1.0,1.0,2.0,16.0


**Churn:**
- A taxa média de churn (abandono) é de aproximadamente 16.8%, indicando que cerca de 1 em cada 6 clientes pode estar abandonando a plataforma.
- A maioria dos clientes não abandonou (mediana e percentis inferiores são 0), sugerindo que a maioria dos clientes permanece ativa.

**Tenure:**
- A média de tenure (tempo de permanência) é negativa (-169.93), o que pode indicar um erro nos dados ou uma interpretação errada da variável. Se for um erro de dados, pode ser necessário revisar a coleta ou processamento dos dados.
- O desvio padrão é alto (1331), sugerindo uma grande variação no tempo de permanência dos clientes.
- A mediana é de 9 dias, e o 75º percentil é de 16 dias, indicando que muitos clientes permanecem por um curto período.

**NumberOfDeviceRegistered:**
- Em média, os clientes registram cerca de 3.69 dispositivos.
- O desvio padrão é baixo (1.03), indicando pouca variação.
- A maioria dos clientes registra entre 3 e 4 dispositivos.

**SatisfactionScore:**
- A pontuação média de satisfação é extremamente alta (11053.48), o que pode indicar um erro nos dados ou uma escala de avaliação não convencional.
- O desvio padrão é muito alto (79946.76), sugerindo uma grande variação nas pontuações de satisfação.
- A mediana é de apenas 3, enquanto o valor máximo é extremamente alto (589314), o que pode indicar outliers ou erros nos dados.

**Complain:**
- A média de reclamações é de aproximadamente 0.29, sugerindo que cerca de 29% dos clientes registraram alguma reclamação.
- A maioria dos clientes não registrou reclamações (mediana e percentis inferiores são 0).

**CouponUsed:**
- Em média, os clientes usam cerca de 1.77 cupons.
- O desvio padrão é de 1.89, indicando uma variação moderada.
- A maioria dos clientes usa entre 1 e 2 cupons.

In [9]:
df_clientes_limpo[categoricos2].describe().T

,count,unique,top,freq
PreferredLoginDevice,4372,4,Mobile Phone,2114
PreferredPaymentMode,4218,7,Debit Card,1719
Gender,4372,2,Male,2656
MaritalStatus,4207,3,Married,2240
NomeCustomer,4372,4136,Igor Barbosa,3


**PreferredLoginDevice (Dispositivo Preferido de Login):**
- O Mobile Phone é o dispositivo preferido por 2115 clientes, representando aproximadamente 48% do total.
- Existem 4 tipos de dispositivos usados para login, mas o Mobile Phone é claramente dominante, sugerindo que a maioria dos clientes prefere acessar o e-commerce por dispositivos móveis.
- A experiência do usuário em dispositivos móveis deve ser uma prioridade para o e-commerce, incluindo design responsivo, velocidade de carregamento e facilidade de navegação.

**PreferredPaymentMode (Modo de Pagamento Preferido):**
- O Debit Card é o método de pagamento mais utilizado, com 1720 clientes preferindo essa opção.
- Existem 7 métodos de pagamento disponíveis, indicando que o e-commerce oferece uma boa diversidade de opções para atender às necessidades dos clientes.
- Como o cartão de débito é amplamente preferido, garantir a segurança e a eficiência das transações com esse método pode melhorar a satisfação do cliente. Além disso, explorar incentivos para outros métodos pode equilibrar a preferência.

**Gender (Gênero):**
- A maioria dos clientes é do gênero Masculino, com 2512 registros (cerca de 57% do total).
- Existem 4 categorias de gênero registradas, indicando que a plataforma atende a uma base diversificada de clientes.
- Campanhas de marketing podem ser direcionadas ao público masculino, mas também é importante considerar estratégias inclusivas para atrair outros gêneros.

**MaritalStatus (Estado Civil):**
- A maioria dos clientes é Casada, com 2240 registros (aproximadamente 53% do total).
- Existem apenas 3 categorias de estado civil registradas.
- Ofertas ou campanhas específicas para casais ou famílias podem ser eficazes, considerando que mais da metade dos clientes está casada.

In [10]:
df_vendas_limpo[categoricos].describe().T

,count,unique,top,freq
StockCode,397884,3665,85123A,2035
Description,397884,3877,WHITE HANGING HEART T-LIGHT HOLDER,2028
InvoiceDate,397884,17282,2011-11-14 15:27:00,542
Country,397884,37,United Kingdom,354321


**StockCode (Código do Estoque):**
- Existem 3665 códigos únicos de estoque, indicando uma ampla diversidade de produtos disponíveis no e-commerce.
- O código de estoque mais frequente é 85123A, com 2035 vendas registradas.
- O produto associado ao código 85123A é popular e pode ser promovido em campanhas de marketing ou destacado em ofertas especiais. Além disso, analisar os motivos por trás de sua alta demanda pode ajudar a replicar o sucesso em outros produtos.

**Description (Descrição do Produto):**
- Há 3877 descrições únicas, o que está alinhado com a diversidade de produtos no estoque.
- O item mais vendido é o WHITE HANGING HEART T-LIGHT HOLDER, com 2028 vendas. Este produto parece ser um dos favoritos dos clientes.
- Esse produto pode ser usado como referência para identificar tendências ou preferências dos consumidores (como itens decorativos). Além disso, é uma oportunidade para criar estratégias de upselling ou cross-selling com produtos similares.

**InvoiceDate (Data da Fatura):**
- Há 17.282 datas únicas registradas, indicando que as vendas ocorreram em diferentes momentos ao longo do período analisado.
- A data mais frequente é 2011-11-14 às 15:27, com 542 transações realizadas nesse momento específico.
- Esse pico pode indicar sazonalidade ou eventos especiais que impulsionaram as vendas nesse dia. É importante investigar se houve promoções ou campanhas específicas nessa data para replicar o sucesso em outros períodos.

**Country (País):**
- O e-commerce realizou vendas para clientes em 37 países, mostrando uma presença internacional significativa.
- O Reino Unido lidera as vendas, com 354.321 transações (cerca de 89% do total).
- Embora o Reino Unido seja o principal mercado, há potencial para expandir a presença nos outros países. Investir em estratégias localizadas para os mercados internacionais pode aumentar a base de clientes fora do Reino Unido.

In [11]:
df_clientes_limpo.head()

,CustomerID,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,NumberOfDeviceRegistered,SatisfactionScore,MaritalStatus,Complain,CouponUsed,NomeCustomer
0,17850.0,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0,Srta. Amanda Sousa
1,13047.0,1,NaN,Phone,UPI,Male,4.0,3,Single,1,0.0,Joaquim Câmara
2,12583.0,1,NaN,Phone,Debit Card,Male,4.0,3,Single,1,0.0,Cauã Rocha
3,13748.0,1,0.0,Phone,Debit Card,Male,4.0,5,Single,0,0.0,Sr. Otávio Andrade
4,15100.0,1,0.0,Phone,CC,Male,3.0,5,Single,0,1.0,Caleb Cunha


In [12]:
df_clientes_limpo.shape

(4372, 12)

In [13]:
df_vendas_limpo.shape

(397884, 8)

## **Distribuição dos dados**

In [14]:
df_clientes_limpo.loc[df_clientes_limpo['SatisfactionScore'] == 589314, 'SatisfactionScore'] = 5
print(df_clientes_limpo['SatisfactionScore'].value_counts())

SatisfactionScore
3    1286
5     903
1     897
4     839
2     447
Name: count, dtype: int64


In [17]:
df_vendas_limpo.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [18]:
df_vendas_limpo['InvoiceDate'].value_counts()

InvoiceDate
2011-11-14 15:27:00    542
2011-11-28 15:54:00    533
2011-12-05 17:17:00    529
2011-11-23 13:39:00    443
2011-10-31 14:09:00    435
                      ... 
2010-12-01 09:00:00      1
2010-12-01 08:35:00      1
2011-10-07 13:57:00      1
2010-12-01 10:37:00      1
2011-06-10 16:47:00      1
Name: count, Length: 17282, dtype: int64